In [1]:
! pip install keras==2.2.5

     |████████████████████████████████| 336 kB 16.3 MB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires keras<2.8,>=2.7.0rc0, but you have keras 2.2.5 which is incompatible.


In [2]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

![Fig7-1-1-1](https://user-images.githubusercontent.com/44194558/142806476-8a492860-463d-4be6-8f0c-53beabdface2.PNG)

<br/>

![Fig7-1-1-3](https://user-images.githubusercontent.com/44194558/142806630-ab8f2fdb-c033-43a7-9ab1-011726ce16f0.PNG)

<br/>

* Depthwise : channel wise nxn spatial convolution (5개의 채널 - 5 spatial convolution)

* Pointwise : 1x1 convolution to change the dim

일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러개 필요함
 - 다양한 입력 소스(Source)에서 전달된 데이터를 다른 종류의 신경망 층을 사용하여 처리하고 합침
 - 가능한 모든 종류의 입력 데이터를 동시에 사용해서 정확한 하나의 모델을 학습


![Fig7-1-3](https://user-images.githubusercontent.com/44194558/142806779-762b48a4-fa18-45b8-8bed-638bdee8a5b5.PNG)

<br/>

![Fig7-1-4](https://user-images.githubusercontent.com/44194558/142806819-7d817c13-ee63-4ab8-a709-275c833ab037.PNG)

## Multi-tasking


입력 데이터에서 여러 개의 타깃 속성을 예측
- ex) 소설이나 짧은 글이 있을 때 장르별로 분류 (로맨스, 스릴러 등) + 글을 쓴 대략의 시대를 예측

- 2 개의 모델을 개별적으로 훈련 가능 (장르 예측 모델, 시대 예측 모델)

But 두 속성은 통계적으로 독립이지 않기 때문에 장르와 시대를 동시에 예측하는 것이 보다 나은 성능을 가져올 수 있음 (2개의 출력/head를 가짐)

- 장르와 시대의 상관관계를 고려하여 보다 정확하고 풍부한 representation을 학습
- 소설의 시대를 고려하여 장르 공간에서 정확하고 풍부한 표현 학습 (vice versa)

![image](https://user-images.githubusercontent.com/44194558/142807253-619a80fd-8676-4020-96df-6aab0e5303d9.png)

최근의 많은 신경망 모델은 비선형 네트워크 topology를 필요로 함
- directed acyclic graphs
- inception network (입력은 나란히 놓인 여러 개의 합성곱 층을 거쳐 하나의 텐서로 출력이 합쳐짐)


![Fig7-1-5-3](https://user-images.githubusercontent.com/44194558/142807449-8b8a2f1e-ede6-40c3-815b-b0090a8cfe3a.PNG)

<br/>


![Fig7-1-7](https://user-images.githubusercontent.com/44194558/142807488-e4325e58-b26f-4535-b57f-5f92b02d4619.PNG)

Resnet, Transformer같이 잔차 연결을 추가하는 경향도 존재
- 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더해 아래층의 representation이 소실되지 않고 네트워크의 위쪽으로 전달될 수 있게 함
- 하위 층에서 학습한 정보의 손실 방지



![Fig7-1-8](https://user-images.githubusercontent.com/44194558/142807753-1d806059-2adc-42fc-b513-e2a6ce5a5024.PNG)

## Functional API

입력 텐서와 출력 텐서만 가지고 Model 객체를 생성
- 케라스는 input_tensor에서 output_tensor로 가는 데 필요한 모든 층을 추출
- 층들을 모아 그래프 데이터 구조인 Model 객체를 생성
- 관련되지 않은 입력과 출력으로 모델을 생성하면 RuntimeError 발생



In [4]:
from tensorflow.keras import Input, layers

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

In [8]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

# Sequential
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Functional
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Keras가 출력 텐서에서 input_3의 텐서로 접근할 수 없어 에러 발생
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)

ValueError: ignored

In [10]:
import numpy as np

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 2s 3ms/step - loss: 11.8970
Epoch 2/10
8/8 [==============================] - 0s 2ms/step - loss: 12.8434
Epoch 3/10
8/8 [==============================] - 0s 3ms/step - loss: 14.6306
Epoch 4/10
8/8 [==============================] - 0s 2ms/step - loss: 17.4011
Epoch 5/10
8/8 [==============================] - 0s 2ms/step - loss: 21.0335
Epoch 6/10
8/8 [==============================] - 0s 2ms/step - loss: 25.2580
Epoch 7/10
8/8 [==============================] - 0s 2ms/step - loss: 29.8442
Epoch 8/10
8/8 [==============================] - 0s 2ms/step - loss: 34.8750
Epoch 9/10
8/8 [==============================] - 0s 2ms/step - loss: 40.4258
Epoch 10/10
32/32 [==============================] - 0s 2ms/step - loss: 50.6004


## Multi-input models

함수형 API는 다중 입력 모델을 만드는 데 사용 가능
- 서로 다른 입력 branch를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용
- 텐서를 더하거나 이어 붙이는 방식 (Element wise sum, concatenation)
- keras.layers.add, keras.layers.concatenate 등 사용



![Fig7-1-16](https://user-images.githubusercontent.com/44194558/142808493-caf3de1a-a8b1-41d1-a660-36adf7e2d222.PNG)

<br/>

질문-응답 모델은 2개의 입력을 가짐
- 자연어 질문 + 답변에 필요한 정보가 담겨 있는 텍스트 (질문에 대한 답 출력)


In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 텍스트
text_input = Input(shape=(None, ), dtype='int32', name='text')  # 텍스트 입력 : 길이가 정해지지 않은 정수 시퀀스 / (None, None)
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)  # 크기가 64인 벡터의 시퀀스로 임베딩 / (None, None, 64)
encoded_text = layers.LSTM(32)(embedded_text)  # 인코딩 / (None, 32)

# 질문
question_input = Input(shape=(None, ), dtype='int32', name='question')  # (None, None)
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)  # (None, None, 32)
encoded_question = layers.LSTM(16)(embedded_question)  # (None, None, 16)

# 합치기
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)  # 인코딩된 질문, 텍스트를 연결, axis의 기본값은 -1 / (None, 32 + 16)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)  # 질문, 텍스트 2개의 입력을 받아 answer 출력
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [12]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
____________________________________________________________________________________________

In [13]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)

model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)  # 입력의 이름을 설정해야 함

Epoch 1/10
8/8 [==============================] - 5s 100ms/step - loss: 0.0000e+00 - acc: 0.0630
Epoch 2/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 4/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 5/10
8/8 [==============================] - 1s 98ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 6/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 7/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 8/10
8/8 [==============================] - 1s 98ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 9/10
8/8 [==============================] - 1s 97ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Epoch 10/10
8/8 [==============================] - 1s 96ms/step - loss: 0.0000e+00 - acc: 0.0000e+00
Ep

In [15]:
print(text.shape)
print(question.shape)
print(answer.shape)

(1000, 100)
(1000, 100)
(None, 500)


넘파이 배열의 리스트를 주입하거나 입력 이름과 넘파이 배열로 이루어진 딕셔너리를 모델의 입력으로 제공

## Multi-output models

데이터에 있는 여러 속성을 동시에 예측하는 네트워크
- 소셜 미디어에서 사용자의 포스트를 입력받아 사용자의 나이, 성별, 소득 수준을 예측

네트워크 출력 마다 다른 손실 함수를 지정해야 함
- 손실들을 하나의 값으로 합쳐야 함
- compile 메서드에 리스트/딕셔너리를 사용하여 출력마다 다른 손실을 지정할 수 있음
- 계산된 손실값들은 전체 손실 하나로 합산되고, 훈련을 통해 최소화됨

![Fig7-1-25](https://user-images.githubusercontent.com/44194558/142809447-b9aa0fd0-0979-455b-89ae-c68a4544cb5f.PNG)

In [16]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)

x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 다중 출력
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [18]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    32000128    embedding_2[0][0]                
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
____________________________________________________________________________________________

In [19]:
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

model.compile(optimizer='rmsprop', loss={'age': 'mse',
                                         'income': 'categorical_crossentropy',
                                         'gender': 'binary_crossentropy'})

손실값들의 불균형 정도가 크면 개별 손실이 가장 큰 작업에 치우쳐 최적화할 가능성이 높음 (다른 작업들이 손해를 보게 됨)

- 개별 손실이 최종 손실에 기여하는 수준을 지정 가능 (손실값의 스케일이 다를 때 유효)
- ex) 나이 regression의 MSE는 3~5, 성별 분류의 cross entropy는 0.1 정도일때 MSE 손실에 0.25의 가중치, cross entropy에 10의 가중치를 부여하여 균형을 맞춤

In [20]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

## Directed acyclic graphs of layers

함수형 API를 사용하여 내부 토폴로지가 복잡한 네트워크를 생성
- 비순환 = 원형 X (텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없음)
- 만들 수 있는 루프는 순환 층의 내부에 있는 것 뿐

인셉션 모듈과 잔차 연결이 가장 유명
- 1x1 합성곱 (pointwise convolution)은 인셉션 모듈의 주요 특징 중 하나
- 채널 방향의 특성 학습과 공간 방향의 특성 학습을 분리하는데 유용
- 채널이 공간 방향으로 상관관계가 크고 채널 간에는 독립적일 때 유용 ex) Mobilenet, Xception

![Fig7-1-37](https://user-images.githubusercontent.com/44194558/142810960-c0942392-0de4-4569-b549-00bc03e044de.PNG)